# Parallel questions study
In this notebook, we carry out the study of the parallel questions related to the influence of movies on baby names, therefore conduction a global analysis.

In [18]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy import stats
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

In [19]:
folder_processed_data_path = './data/processed_data/'

# Dataset containing month of release
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)
display(movie_df)

# Dataset containing p_value
name_by_movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
name_by_movie_df.set_index(['wiki_ID'], inplace=True)
display(name_by_movie_df)

# Dataset containing movie genre
movie_genres_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_genres_df.csv'))
movie_genres_df.set_index(['wiki_ID'], inplace=True)
display(movie_genres_df)

,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,
975900,Ghosts of Mars,2001,8.0,14010832.0,56880,4.9
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000,2.0,NaN,69,6.0
28463795,Brun bitter,1988,NaN,NaN,40,5.6
9363483,White Of The Eye,1987,NaN,NaN,2891,6.1
261236,A Woman in Flames,1983,NaN,NaN,623,5.9
...,...,...,...,...,...,...
35228177,Mermaids: The Body Found,2011,3.0,NaN,1711,4.6
34980460,Knuckle,2011,1.0,NaN,3192,6.8
9971909,Another Nice Mess,1972,9.0,NaN,111,5.8


,char_words,gender,order,p_value,slope_change,t_stat
wiki_ID,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN
3217,Linda,F,7.0,0.676072,-0.000675,0.429187
3217,Henry,M,4.0,0.068422,-0.002435,2.019954
3217,Duke,M,4.0,0.582585,0.000108,-0.566260
3217,Warrior,M,9.0,NaN,NaN,NaN
...,...,...,...,...,...,...
37478048,Ajay,M,9.0,0.436957,-0.000126,0.806658
37501922,Murphy,F,3.0,0.234444,0.000354,-1.257988
37501922,Hunter,M,1.0,0.000021,-0.035578,7.051709


,genre
wiki_ID,
330,Comedy-drama
330,Drama
3217,Action
3217,Comedy
3217,Time travel
...,...
37476824,Crime Comedy
37476824,Caper story
37476824,Crime Fiction


### Question 1: Month of release

In [20]:
# First, aggregate dataframe with p_value table with dataframe containing release month 
name_by_movie_aggregate_df = name_by_movie_df.merge(movie_df, how='left', left_on='wiki_ID', right_on='wiki_ID')
display(name_by_movie_aggregate_df)

,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Linda,F,7.0,0.676072,-0.000675,0.429187,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Henry,M,4.0,0.068422,-0.002435,2.019954,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Duke,M,4.0,0.582585,0.000108,-0.566260,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Warrior,M,9.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...
37478048,Ajay,M,9.0,0.436957,-0.000126,0.806658,Mr. Bechara,1996,NaN,NaN,395,5.4
37501922,Murphy,F,3.0,0.234444,0.000354,-1.257988,Terminal Bliss,1992,NaN,NaN,245,4.4
37501922,Hunter,M,1.0,0.000021,-0.035578,7.051709,Terminal Bliss,1992,NaN,NaN,245,4.4


In [21]:
summer = [6.0, 7.0, 8.0]
fall = [9.0,10.0,11.0]
winter = [12.0,1.0,2.0]
spring = [3.0,4.0,5.0]
summer_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(summer)]
fall_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(fall)]
winter_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(winter)]
spring_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(spring)]


display(summer_movies_df)
display(fall_movies_df)
display(winter_movies_df)
display(spring_movies_df)

,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
3746,Deckard,M,0.0,NaN,NaN,NaN,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Eldon,M,8.0,0.653938,-0.000105,0.460773,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Lewis,M,12.0,0.327638,-0.000698,1.024419,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Bear,M,11.0,NaN,NaN,NaN,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Leon,M,7.0,0.469966,0.000525,-0.748319,Blade Runner,1982,6.0,33139618.0,804384,8.1
...,...,...,...,...,...,...,...,...,...,...,...,...
36699915,Luke,M,5.0,0.846639,0.001421,-0.198023,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7
36699915,Underwood,M,1.0,NaN,NaN,NaN,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7
36699915,Chase,F,2.0,0.149005,0.011448,-1.551732,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7


,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Linda,F,7.0,0.676072,-0.000675,0.429187,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Henry,M,4.0,0.068422,-0.002435,2.019954,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Duke,M,4.0,0.582585,0.000108,-0.566260,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Warrior,M,9.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...
37322106,Major,M,0.0,0.080101,-0.002548,1.927686,Jab Tak Hai Jaan,2012,11.0,NaN,58012,6.7
37373877,Beth,F,5.0,0.425923,-0.000270,0.826799,Crazy Eights,2006,10.0,NaN,3338,3.8
37373877,Patterson,F,5.0,NaN,NaN,NaN,Crazy Eights,2006,10.0,NaN,3338,3.8


,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
3837,Lamarr,M,3.0,0.765429,0.000046,-0.305851,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Van,M,11.0,0.249160,-0.000456,1.216756,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Bart,M,0.0,0.861123,0.000158,-0.179090,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Lyle,M,6.0,0.879211,-0.000105,0.155542,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Buddy,M,18.0,0.965905,0.000018,-0.043728,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...
36956792,Kid,M,18.0,NaN,NaN,NaN,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4
36956792,Charlie,M,5.0,0.000208,-0.006215,5.427450,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4
36956792,Beach,M,18.0,NaN,NaN,NaN,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4


,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
4560,Morrison,M,19.0,NaN,NaN,NaN,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Edward,M,3.0,0.713319,-0.000845,0.377036,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Campbell,M,5.0,0.113180,-0.000474,1.721204,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Murron,F,1.0,NaN,NaN,NaN,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,William,M,0.0,0.006095,-0.015277,3.384440,Braveheart,1995,5.0,211409945.0,1072580,8.3
...,...,...,...,...,...,...,...,...,...,...,...,...
36814246,Girl,F,4.0,NaN,NaN,NaN,Eraserhead,1977,3.0,7000000.0,124128,7.3
36814246,Mary,F,1.0,0.017282,-0.041302,2.799750,Eraserhead,1977,3.0,7000000.0,124128,7.3
36814246,Beautiful,F,4.0,NaN,NaN,NaN,Eraserhead,1977,3.0,7000000.0,124128,7.3


In [22]:
summer_movies_df.index.unique()

Index([    3746,     3947,     4231,     4726,     4727,     4728,     4729,
           4730,     8481,     9979,
       ...
       36306987, 36329343, 36354051, 36354224, 36422681, 36448415, 36478252,
       36566804, 36617100, 36699915],
      dtype='int64', name='wiki_ID', length=4096)

In [23]:
prop_summer = len(summer_movies_df[summer_movies_df['p_value']<0.1])/len(summer_movies_df['p_value'])
display(prop_summer)
prop_fall = len(fall_movies_df[fall_movies_df['p_value']<0.1])/len(fall_movies_df['p_value'])
display(prop_fall)
prop_winter = len(winter_movies_df[winter_movies_df['p_value']<0.1])/len(winter_movies_df['p_value'])
display(prop_winter)
prop_spring = len(spring_movies_df[spring_movies_df['p_value']<0.1])/len(spring_movies_df['p_value'])
display(prop_spring)

0.13600104190407972

0.14431685722080306

0.13875639832480224

0.13644032637256256

Faire test statistique pour dire que different ? Je dirai que oui et on le fait en dessous

In [24]:
from scipy.stats import chi2_contingency

# Organize the data into a contingency table
observed_data = [
    [len(summer_movies_df[summer_movies_df['p_value'] < 0.1]), len(summer_movies_df['p_value'])],
    [len(fall_movies_df[fall_movies_df['p_value'] < 0.1]), len(fall_movies_df['p_value'])],
    [len(winter_movies_df[winter_movies_df['p_value'] < 0.1]), len(winter_movies_df['p_value'])],
    [len(spring_movies_df[spring_movies_df['p_value'] < 0.1]), len(spring_movies_df['p_value'])]
]

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(observed_data)

# Print the results
print("Chi-squared value:", chi2)
print("P-value:", p)

Chi-squared value: 10.399003008260278
P-value: 0.015461904797585156


H0 : The proportions are all equal 

We can reject the null hypothesis at the 5% significance level.

### Question 2: Movie Genre has an impact ?

In [25]:
# First, aggregate dataframe with p_value table with dataframe containing movie genre
# Outer merge required in order to obtain for each name of each film, all the possible genre it can be associated to 
movie_genre_aggregate_df = name_by_movie_df.merge(movie_genres_df, how='outer', left_on='wiki_ID', right_on='wiki_ID')
movie_genre_aggregate_df.head(25)

,char_words,gender,order,p_value,slope_change,t_stat,genre
wiki_ID,,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN,Action
3217,Gold,NaN,6.0,NaN,NaN,NaN,Comedy
3217,Gold,NaN,6.0,NaN,NaN,NaN,Time travel
3217,Gold,NaN,6.0,NaN,NaN,NaN,Black comedy
3217,Gold,NaN,6.0,NaN,NaN,NaN,Zombie Film
3217,Gold,NaN,6.0,NaN,NaN,NaN,Horror Comedy
3217,Gold,NaN,6.0,NaN,NaN,NaN,Action/Adventure
3217,Gold,NaN,6.0,NaN,NaN,NaN,Costume drama
3217,Gold,NaN,6.0,NaN,NaN,NaN,Stop motion


In [26]:
name_by_genre_significant_df = movie_genre_aggregate_df.groupby('genre').apply(lambda x: x[x['p_value'] < 0.1])
display(name_by_genre_significant_df)

char_words gender  order   p_value  slope_change  \
genre       wiki_ID                                                     
Absurdism   19701           Tim      M    1.0  0.001787     -0.010096   
            46505        Victor      M   11.0  0.070347     -0.003045   
            46505           Ted      M    0.0  0.046833     -0.001105   
            46505        Johnny      M   10.0  0.039946     -0.002623   
            46505         Roger      M    3.0  0.066845     -0.004609   
...                         ...    ...    ...       ...           ...   
Zombie Film 31633054       Andy      M    0.0  0.077429     -0.001991   
            33432215     Cassie      F    3.0  0.061421     -0.001099   
            33432215      Sarah      F    7.0  0.000060     -0.020843   
            33432215       Mack      M    4.0  0.023270     -0.001138   
            33432215      Henry      M    0.0  0.020617     -0.009615   

                        t_stat        genre  
genre       wiki_ID                          
Absurdism   19701     4.090696    Absurdism  
            46505     2.003778    Absurdism  
            46505     2.238396    Absurdism  
            46505     2.328911    Absurdism  
            46505     2.033517    Absurdism  
...                        ...          ...  
Zombie Film 31633054  1.947633  Zombie Film  
            33432215  2.082602  Zombie Film  
            33432215  6.283305  Zombie Film  
            33432215  2.633292  Zombie Film  
            33432215  2.701090  Zombie Film  

[105588 rows x 7 columns]

In [27]:
# Compute proportion of impacted names by genre
name_by_genre_prop_significant_df = movie_genre_aggregate_df.groupby('genre').apply(lambda x: pd.Series({
        'prop_signif_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'prop_non_signi': (x['p_value'] > 0.1).sum()/len(x['p_value']),
        'prop_nan': (x['p_value'].isna()).sum()/len(x['p_value']),
        'avg_t_value_per_genre': x['t_stat'].mean(),
        'avg_slope_change' : x['slope_change'].mean()
    }))
display(name_by_genre_prop_significant_df)


,prop_signif_per_genre,prop_non_signi,prop_nan,avg_t_value_per_genre,avg_slope_change
genre,,,,,
Absurdism,0.132432,0.570270,0.297297,0.500326,-0.000527
Acid western,0.187500,0.562500,0.250000,0.843112,-0.002524
Action,0.117510,0.469206,0.413283,0.457436,-0.000264
Action Comedy,0.124517,0.505792,0.369691,0.431969,-0.000097
Action Thrillers,0.128478,0.517004,0.354517,0.421422,-0.000294
...,...,...,...,...,...
World History,0.000000,0.000000,1.000000,NaN,NaN
World cinema,0.067656,0.379661,0.552683,0.184046,0.000080
Wuxia,0.032558,0.158140,0.809302,0.390231,-0.000327


#### Analysis looking at time effects

In [28]:
# Need to merge datasets containing "p_value" (name_by_movie_df), "movie_genre" (movie_genres_df), "release_date" (movie_df)
# => aggregate "name_by_movie_aggregate_df" with "movie_genres_df"
movie_genre_aggregate_with_years_df = movie_genre_aggregate_df.merge(movie_df, how='left', left_on='wiki_ID', right_on='wiki_ID')
display(movie_genre_aggregate_with_years_df)

,char_words,gender,order,p_value,slope_change,t_stat,genre,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN,Action,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Gold,NaN,6.0,NaN,NaN,NaN,Comedy,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Gold,NaN,6.0,NaN,NaN,NaN,Time travel,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Gold,NaN,6.0,NaN,NaN,NaN,Black comedy,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Gold,NaN,6.0,NaN,NaN,NaN,Zombie Film,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37241569,NaN,NaN,NaN,NaN,NaN,NaN,Action,Cold War,2012,11.0,NaN,5033,6.6
37476824,NaN,NaN,NaN,NaN,NaN,NaN,Comedy,I Love New Year,2011,NaN,NaN,876,3.4
37476824,NaN,NaN,NaN,NaN,NaN,NaN,Crime Comedy,I Love New Year,2011,NaN,NaN,876,3.4


In [29]:
signi_names_per_genre_per_year_df = movie_genre_aggregate_with_years_df.groupby(['genre','year']).apply(lambda x: x[x['p_value'] < 0.1])
signi_names_per_genre_per_year_df.head(30)

char_words gender  order   p_value  slope_change  \
genre     year wiki_ID                                                    
Absurdism 1964 248601      George      M    2.0  0.026640     -0.011959   
               248601        John      M    0.0  0.047794      0.073648   
          1974 19701          Tim      M    1.0  0.001787     -0.010096   
          1978 75261       Robert      M    9.0  0.004233     -0.053751   
               75261         Dave      M   19.0  0.030485     -0.001459   
               75261      Barbara      F   15.0  0.004086     -0.018562   
               75261         Kent      M    8.0  0.022216     -0.002411   
               75261      Douglas      M    NaN  0.088331     -0.010605   
               75261       Donald      M    6.0  0.011954     -0.010231   
               75261          Guy      M   39.0  0.078059     -0.001660   
               75261         Dean      M    2.0  0.009035     -0.008901   
               75261         John      M    0.0  0.005100     -0.060190   
          1980 46505       Victor      M   11.0  0.070347     -0.003045   
               46505          Ted      M    0.0  0.046833     -0.001105   
               46505       Johnny      M   10.0  0.039946     -0.002623   
               46505        Roger      M    3.0  0.066845     -0.004609   
               4558294     Donald      M    7.0  0.062334     -0.007153   
          1982 1763713        Ted      M    8.0  0.079545     -0.001105   
          1984 983849      Martin      M    NaN  0.056623     -0.004328   
          1987 75222     Princess      F    3.0  0.082302      0.000706   
               1716921     Brenda      F    0.0  0.029961     -0.005886   
               1716921      Jerry      M   51.0  0.077471     -0.003322   
               1716921      Karen      F    8.0  0.009398     -0.015997   
          1991 451109       Julie      F    1.0  0.033500     -0.009879   
               451109      Robert      M    7.0  0.089618      0.015846   
               1048388       Sean      M    NaN  0.031758      0.021310   
               1048388       Lane      F    2.0  0.020485     -0.005893   
               1048388   Scarlett      F    2.0  0.081415     -0.000775   
               1048388     Harley      M    0.0  0.082786     -0.004870   
          1992 77744         Kane      M    2.0  0.094734     -0.000888   

                          t_stat      genre                         mov_name  \
genre     year wiki_ID                                                         
Absurdism 1964 248601   2.557438  Absurdism               A Hard Day's Night   
               248601  -2.226790  Absurdism               A Hard Day's Night   
          1974 19701    4.090696  Absurdism  Monty Python and the Holy Grail   
          1978 75261    3.591418  Absurdism  National Lampoon's Animal House   
               75261    2.481634  Absurdism  National Lampoon's Animal House   
               75261    3.611592  Absurdism  National Lampoon's Animal House   
               75261    2.659263  Absurdism  National Lampoon's Animal House   
               75261    1.869884  Absurdism  National Lampoon's Animal House   
               75261    3.005882  Absurdism  National Lampoon's Animal House   
               75261    1.942873  Absurdism  National Lampoon's Animal House   
               75261    3.162738  Absurdism  National Lampoon's Animal House   
               75261    3.485338  Absurdism  National Lampoon's Animal House   
          1980 46505    2.003778  Absurdism                        Airplane!   
               46505    2.238396  Absurdism                        Airplane!   
               46505    2.328911  Absurdism                        Airplane!   
               46505    2.033517  Absurdism                        Airplane!   
               4558294  2.074061  Absurdism               The Blues Brothers   
          1982 1763713  1.931787  Absurdism          Airplane II: The Sequel   
          1984 983849 

In [30]:
# Compute proportion of impacted names by genre by year
name_by_genre_by_year_prop_significant_df = movie_genre_aggregate_with_years_df.groupby(['genre','year']).apply(lambda x: pd.Series({
        'prop_signif_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change': x['slope_change'].mean()
    }))
display(name_by_genre_by_year_prop_significant_df)
name_by_genre_by_year_prop_significant_df.head(30)

prop_signif_per_genre  avg_slope_change
genre       year                                         
Absurdism   1929               0.000000               NaN
            1930               0.000000               NaN
            1932               0.000000          0.000171
            1938               0.000000               NaN
            1940               0.000000               NaN
...                                 ...               ...
Zombie Film 2008               0.168675         -0.000864
            2009               0.093023          0.001817
            2010               0.172414         -0.001540
            2011               0.307692         -0.003649
            2012               0.000000               NaN

[13832 rows x 2 columns]

prop_signif_per_genre  avg_slope_change
genre     year                                         
Absurdism 1929               0.000000               NaN
          1930               0.000000               NaN
          1932               0.000000          0.000171
          1938               0.000000               NaN
          1940               0.000000               NaN
          1941               0.000000               NaN
          1950               0.000000         -0.000357
          1959               0.000000               NaN
          1964               0.333333          0.014922
          1965               0.000000          0.001620
          1966               0.000000               NaN
          1968               0.000000               NaN
          1969               0.000000               NaN
          1970               0.000000               NaN
          1971               0.000000          0.000200
          1973               0.000000               NaN
          1974               0.071429         -0.004156
          1976               0.000000          0.000296
          1977               0.000000         -0.006322
          1978               0.209302         -0.004627
          1979               0.000000          0.000167
          1980               0.135135         -0.002954
          1982               0.052632         -0.002621
          1983               0.000000         -0.000220
          1984               0.071429         -0.000304
          1987               0.181818         -0.002599
          1989               0.000000         -0.001037
          1990               0.000000         -0.000010
          1991               0.187500          0.001991
          1992               0.227273         -0.002618

### Question 3: Attendence/popularity + ratings

In [31]:
# The dataframe "name_by_movie_aggregate_df" already contains the wanted caracteristics
display(name_by_movie_aggregate_df)
name_by_movie_aggregate_df['numVotes'].max()

#Proportion of the film that had an influence in data segmented by number of votes

prop_0_10k = len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] < 10000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]) / len(name_by_movie_aggregate_df[name_by_movie_aggregate_df['numVotes'] < 10000])

print(f"Proportion of movies with numVotes < 10k and p_value < 0.1: {prop_0_10k :.3%}")

prop_10k_100k = len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 10000) & (name_by_movie_aggregate_df['numVotes'] < 100000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]) / len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 10000) & (name_by_movie_aggregate_df['numVotes'] < 100000)])

print(f"Proportion of movies with numVotes in [10k-100k] and p_value < 0.1: {prop_10k_100k :.3%}")

prop_100k_1M = len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 100000) & (name_by_movie_aggregate_df['numVotes'] < 1000000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]) / len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 100000) & (name_by_movie_aggregate_df['numVotes'] < 1000000)])

print(f"Proportion of movies with numVotes in [100k-1M] and p_value < 0.1: {prop_100k_1M :.3%}")

prop_greater_1M = len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 1000000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]) / len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 1000000)])

print(f"Proportion of movies with numVotes > 1M and p_value < 0.1: {prop_greater_1M :.3%}")


len(name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 1000000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]['numVotes'].unique())


,char_words,gender,order,p_value,slope_change,t_stat,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,,
3217,Gold,NaN,6.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Linda,F,7.0,0.676072,-0.000675,0.429187,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Henry,M,4.0,0.068422,-0.002435,2.019954,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Duke,M,4.0,0.582585,0.000108,-0.566260,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Warrior,M,9.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...
37478048,Ajay,M,9.0,0.436957,-0.000126,0.806658,Mr. Bechara,1996,NaN,NaN,395,5.4
37501922,Murphy,F,3.0,0.234444,0.000354,-1.257988,Terminal Bliss,1992,NaN,NaN,245,4.4
37501922,Hunter,M,1.0,0.000021,-0.035578,7.051709,Terminal Bliss,1992,NaN,NaN,245,4.4


Proportion of movies with numVotes < 10k and p_value < 0.1: 12.741%
Proportion of movies with numVotes in [10k-100k] and p_value < 0.1: 15.123%
Proportion of movies with numVotes in [100k-1M] and p_value < 0.1: 15.200%
Proportion of movies with numVotes > 1M and p_value < 0.1: 13.737%


50

Assumption: 
-Attendence is estimated by the number of votes
-A threshold of # of votes anove wich we start to study the influence of rating 

Ideas: 

-separate data according to number of votes & then separate data accordimng to rating 

-separate first according to votes and then in the segments of votes separates bad and good reviews

Question 4 : Faire la moyenne

In [16]:
#We segment the data frame according to the number of votes

votes_seg_0_10k = name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] < 10000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]
votes_seg_10k_100k = name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 10000) & (name_by_movie_aggregate_df['numVotes'] < 100000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]
votes_seg_100k_1M = name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 100000) & (name_by_movie_aggregate_df['numVotes'] < 1000000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]
votes_seg_1M_inf = name_by_movie_aggregate_df[(name_by_movie_aggregate_df['numVotes'] > 1000000) & (name_by_movie_aggregate_df['p_value'] < 0.1)]

a = [votes_seg_0_10k['slope_change'].mean(), votes_seg_10k_100k['slope_change'].mean(), votes_seg_100k_1M['slope_change'].mean(), votes_seg_1M_inf['slope_change'].mean()]
index_names = ['0-10k', '10k-100k', '100k-1M', '1M-inf']
results = pd.DataFrame(a, index=index_names,columns = ['avg_slope_change'])
results.index.name = 'Seg_numVotes'
display(results)

,avg_slope_change
Seg_numVotes,
0-10k,-0.001290
10k-100k,-0.001823
100k-1M,-0.002344
1M-inf,-0.000799


In [44]:
name_by_movie_aggregate_df_significant = name_by_movie_aggregate_df[name_by_movie_aggregate_df['p_value'] < 0.1]

#We segment the data frame according to the number of votes

##Calculate the average cahnge of slopes for the different number of vote segments 

numVotes_bins = [0,10000,100000,1000000,np.inf]
segments_numVotes_label = ['0-10000','10000-100000','100000-1000000','1000000+']
name_by_movie_aggregate_df_significant['numVotes_segmented']  = pd.cut(name_by_movie_aggregate_df_significant['numVotes'],numVotes_bins,labels=segments_numVotes_label,right=True)

avg_slopes_numVotes = name_by_movie_aggregate_df_significant.groupby('numVotes_segmented')[['slope_change']].mean()
avg_slopes_numVotes.rename(columns={'slope_change': 'avg_slopes_change'},inplace=True)

display(avg_slopes_numVotes) #Average slope change per segment

avg_magnitude_slopes_change_numVotes = name_by_movie_aggregate_df_significant.groupby('numVotes_segmented')[['slope_change']].apply(lambda x: x.abs().mean())
avg_magnitude_slopes_change_numVotes.rename(columns={'slope_change': 'avg_magnitude_slopes_change'},inplace=True)
display(avg_magnitude_slopes_change_numVotes) #Average of the magnitude of the slope change



C:\Users\thomd\AppData\Local\Temp\ipykernel_10332\1308437321.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_by_movie_aggregate_df_significant['numVotes_segmented']  = pd.cut(name_by_movie_aggregate_df_significant['numVotes'],numVotes_bins,labels=segments_numVotes_label,right=True)


,avg_slopes_change
numVotes_segmented,
0-10000,-0.001290
10000-100000,-0.001823
100000-1000000,-0.002344
1000000+,-0.000799


,avg_magnitude_slopes_change
numVotes_segmented,
0-10000,0.013986
10000-100000,0.010802
100000-1000000,0.009755
1000000+,0.011324


In [42]:
#We segment the data frame according to the rating
#Calculate the average change of slopes for the different rating segements

rating_quantiles = np.quantile(name_by_movie_aggregate_df_significant['averageRating'],[0.25,0.5,0.75])
#display(rating_quantiles)

# display((name_by_movie_aggregate_df_significant['averageRating']<= 5.5).sum()/len(name_by_movie_aggregate_df_significant))

rating_bins = [0,rating_quantiles[0],rating_quantiles[1],rating_quantiles[2],10]
segments_rating_label = ['0-5.5','5.5-6.3','6.3-6.9','6.9-10']
name_by_movie_aggregate_df_significant['rating_segmented']  = pd.cut(name_by_movie_aggregate_df_significant['averageRating'],rating_bins,labels=segments_rating_label,right=True)
avg_slopes_change_rating = name_by_movie_aggregate_df_significant.groupby('rating_segmented')[['slope_change']].mean()
avg_slopes_change_rating.rename(columns = {'slope_change': 'avg_slopes_change'},inplace=True)
display(avg_slopes_change_rating)

avg_magnitude_slopes_change_rating = name_by_movie_aggregate_df_significant.groupby('rating_segmented')[['slope_change']].apply(lambda x : x.abs().mean())
avg_magnitude_slopes_change_rating.rename(columns = {'slope_change':'avg_magnitude_slopes_change'})


C:\Users\thomd\AppData\Local\Temp\ipykernel_10332\2435210583.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_by_movie_aggregate_df_significant['rating_segmented']  = pd.cut(name_by_movie_aggregate_df_significant['averageRating'],rating_bins,labels=segments_rating_label,right=True)


,avg_slopes_change
rating_segmented,
0-5.5,-0.001735
5.5-6.3,-0.001851
6.3-6.9,-0.001360
6.9-10,-0.001422


,avg_magnitude_slopes_change
rating_segmented,
0-5.5,0.011755
5.5-6.3,0.012728
6.3-6.9,0.012693
6.9-10,0.012424


### Question 4: Character Importance in film

In [ ]:
# The dataframe "name_by_movie_df" already contains the wanted caracteristics ("order")
#name_by_order_df = name_by_movie_aggregate_df.groupby("order").apply(lambda x: x[x['p_value'] < 0.1])
#display(name_by_order_df)
name_by_order_df = name_by_movie_df.groupby("order").apply(lambda x: x[x['p_value'] < 0.1])
display(name_by_order_df)

char_words gender  order   p_value  slope_change    t_stat
order wiki_ID                                                            
0.0   4560        William      M    0.0  0.006095     -0.015277  3.384440
      5035           Eric      M    0.0  0.000031     -0.025032  6.755076
      5729         Harold      M    0.0  0.045354     -0.001979  2.256695
      19715        Gracie      F    0.0  0.013734     -0.008358  2.928240
      22751         Julia      F    0.0  0.013218      0.023395 -2.949628
...                   ...    ...    ...       ...           ...       ...
94.0  9834441        Lily      F   94.0  0.000714      0.023868 -4.642400
95.0  20777420     Thomas      M   95.0  0.001181     -0.010917  4.336954
98.0  370064     Anderson      F   98.0  0.001162     -0.003267  4.346455
      25079197      Tyson      M   98.0  0.001493      0.003804 -4.197156
114.0 33028800  Christine      F  114.0  0.009449     -0.001611  3.137582

[19969 rows x 6 columns]

In [51]:
name_by_order_prop_df = name_by_movie_df.groupby("order").apply(lambda x: pd.Series({
        'prop_signif_per_order': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change': x['slope_change'].mean(),
        'total_number_signif_per_order': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_order_prop_df)


,prop_signif_per_order,avg_slope_change,total_number_signif_per_order
order,,,
0.0,0.152382,-0.000224,3723.0
1.0,0.153846,0.000031,2910.0
2.0,0.150620,-0.000290,2187.0
3.0,0.148164,-0.000378,1767.0
4.0,0.144475,-0.000353,1454.0
...,...,...,...
151.0,0.000000,0.001574,0.0
152.0,0.000000,-0.000905,0.0
169.0,0.000000,-0.000067,0.0


#### Does movie genre and caracter role are linked ?

In [52]:
# Does the order influence is impacted by movie genre ? Study of impact due to role importance per movie genre
name_by_order_by_genre_prop_df = movie_genre_aggregate_df.groupby(['order','genre']).apply(lambda x: pd.Series({
        'prop_signif_per_order_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change': x['slope_change'].mean(),
        'total_number_signif_per_order_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_order_by_genre_prop_df)

prop_signif_per_order_per_genre  avg_slope_change  \
order genre                                                                  
0.0   Absurdism                                 0.189873          0.003077   
      Acid western                              0.500000         -0.000975   
      Action                                    0.150447         -0.000131   
      Action Comedy                             0.113402          0.000727   
      Action Thrillers                          0.159269         -0.000243   
...                                                  ...               ...   
302.0 Biographical film                         0.000000               NaN   
      Biography                                 0.000000               NaN   
      Drama                                     0.000000               NaN   
      Period piece                              0.000000               NaN   
      Political drama                           0.000000               NaN   

                         total_number_signif_per_order_per_genre  
order genre                                                       
0.0   Absurdism                                             15.0  
      Acid western                                           4.0  
      Action                                               657.0  
      Action Comedy                                         11.0  
      Action Thrillers                                      61.0  
...                                                          ...  
302.0 Biographical film                                      0.0  
      Biography                                              0.0  
      Drama                                                  0.0  
      Period piece                                           0.0  
      Political drama                                        0.0  

[9390 rows x 3 columns]

### Does the order of a name influence differently according to gender ?
<span style="color:red"> *Prendre seulement les valeur ou p less 0.1 pour faire l'etude des slopes ? Si on les gardes ça va influencer nos moyenne avec des truc pas significantes *</span>

In [62]:
# Calculate the average magnitude of slope change on all the data
name_by_order_by_gender_prop_df = movie_genre_aggregate_df.groupby(['order','gender']).apply(lambda x: pd.Series({
        'prop_signif_per_order_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_magnitude_slope_change': x['slope_change'].abs().mean(),
        'total_number_signif_per_order_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_order_by_gender_prop_df)

# Calculate the average magnitude of slope change on data having a slope change statistically significant

name_by_order_by_gender_prop_aggregate_significant = name_by_movie_aggregate_df_significant.groupby(['order','gender']).apply(lambda x: pd.Series({
        'prop_signif_per_order_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_magnitude_slope_change': x['slope_change'].abs().mean(),
        'total_number_signif_per_order_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_order_by_gender_prop_aggregate_significant)

# Calculate the average of slope change on data having a slope change statistically significant

name_by_order_by_gender_prop_aggregate_sig = name_by_movie_aggregate_df_significant.groupby(['order','gender']).apply(lambda x: pd.Series({
        'prop_signif_per_order_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_magnitude_slope_change': x['slope_change'].mean(),
        'total_number_signif_per_order_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_order_by_gender_prop_aggregate_sig)

prop_signif_per_order_per_genre  avg_magnitude_slope_change  \
order gender                                                                
0.0   F                              0.176309                    0.005117   
      M                              0.147234                    0.004940   
1.0   F                              0.166325                    0.004809   
      M                              0.146448                    0.004309   
2.0   F                              0.164852                    0.004664   
...                                       ...                         ...   
151.0 M                              0.000000                    0.001574   
152.0 F                              0.000000                    0.000905   
169.0 M                              0.000000                    0.000067   
300.0 M                              0.000000                    0.000038   
302.0 M                              0.000000                         NaN   

              total_number_signif_per_order_per_genre  
order gender                                           
0.0   F                                        5336.0  
      M                                       10602.0  
1.0   F                                        6646.0  
      M                                        5978.0  
2.0   F                                        4420.0  
...                                               ...  
151.0 M                                           0.0  
152.0 F                                           0.0  
169.0 M                                           0.0  
300.0 M                                           0.0  
302.0 M                                           0.0  

[197 rows x 3 columns]

prop_signif_per_order_per_genre  avg_magnitude_slope_change  \
order gender                                                                
0.0   F                                   1.0                    0.014784   
      M                                   1.0                    0.015421   
1.0   F                                   1.0                    0.014767   
      M                                   1.0                    0.013695   
2.0   F                                   1.0                    0.013123   
...                                       ...                         ...   
94.0  F                                   1.0                    0.012731   
95.0  M                                   1.0                    0.010917   
98.0  F                                   1.0                    0.003267   
      M                                   1.0                    0.003804   
114.0 F                                   1.0                    0.001611   

              total_number_signif_per_order_per_genre  
order gender                                           
0.0   F                                        1283.0  
      M                                        2392.0  
1.0   F                                        1540.0  
      M                                        1343.0  
2.0   F                                        1015.0  
...                                               ...  
94.0  F                                           2.0  
95.0  M                                           1.0  
98.0  F                                           1.0  
      M                                           1.0  
114.0 F                                           1.0  

[130 rows x 3 columns]

prop_signif_per_order_per_genre  avg_magnitude_slope_change  \
order gender                                                                
0.0   F                                   1.0                   -0.000833   
      M                                   1.0                   -0.002366   
1.0   F                                   1.0                    0.000097   
      M                                   1.0                   -0.001762   
2.0   F                                   1.0                   -0.001191   
...                                       ...                         ...   
94.0  F                                   1.0                    0.011138   
95.0  M                                   1.0                   -0.010917   
98.0  F                                   1.0                   -0.003267   
      M                                   1.0                    0.003804   
114.0 F                                   1.0                   -0.001611   

              total_number_signif_per_order_per_genre  
order gender                                           
0.0   F                                        1283.0  
      M                                        2392.0  
1.0   F                                        1540.0  
      M                                        1343.0  
2.0   F                                        1015.0  
...                                               ...  
94.0  F                                           2.0  
95.0  M                                           1.0  
98.0  F                                           1.0  
      M                                           1.0  
114.0 F                                           1.0  

[130 rows x 3 columns]

### Question 5: Caracter gender in film

In [ ]:
# The dataframe "name_by_movie" has everything we need
name_by_gender_df = name_by_movie_df.groupby('gender').apply(lambda x: x[x['p_value'] < 0.1])
display(name_by_gender_df)

char_words gender  order   p_value  slope_change    t_stat
gender wiki_ID                                                            
F      3947        Barbara      F    8.0  0.063999     -0.003945  2.058789
       4231      Cassandra      F    8.0  0.071099      0.017066 -1.997573
       4231       Jennifer      F    6.0  0.031782     -0.086605  2.458174
       4560       Isabelle      F    2.0  0.000007     -0.008570  7.995228
       4726          Vicki      F    2.0  0.084777     -0.000762  1.894210
...                    ...    ...    ...       ...           ...       ...
M      36956792     Gunner      M   13.0  0.001406     -0.002385  4.232822
       36956792    Charlie      M    5.0  0.000208     -0.006215  5.427450
       37322106      Major      M    0.0  0.080101     -0.002548  1.927686
       37501922     Hunter      M    1.0  0.000021     -0.035578  7.051709
       37501922       John      M    1.0  0.052067     -0.012279  2.177768

[22992 rows x 6 columns]

<span style="color:red"> *Même question, est ce que on fait notre analyse sur les film significant suelement ou pas *</span>

In [67]:
#Average slope change 
name_by_gender_prop_df = name_by_movie_df.groupby("gender").apply(lambda x: pd.Series({
        'prop_signif_per_gender': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change_per_gender': x['slope_change'].mean(),
        'total_number_signif_per_gender': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_gender_prop_df)

#Average magnitude slope change taking significant only
name_by_gender_prop_df = name_by_movie_aggregate_df_significant.groupby("gender").apply(lambda x: pd.Series({
        'prop_signif_per_gender': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_magnitude_slope_change_per_gender': x['slope_change'].abs().mean(),
        'total_number_signif_per_gender': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_gender_prop_df)

,prop_signif_per_gender,avg_slope_change_per_gender,total_number_signif_per_gender
gender,,,
F,0.156183,-0.000154,9793.0
M,0.126010,-0.000378,13199.0


,prop_signif_per_gender,avg_magnitude_slope_change_per_gender,total_number_signif_per_gender
gender,,,
F,1.0,0.012687,9793.0
M,1.0,0.012182,13199.0


#### Does caracter gender and movie genre are linked ?

In [68]:
# Does the gender influence is impacted by movie genre ? Study of impact due to role importance per movie genre
name_by_gender_by_genre_prop_df = movie_genre_aggregate_df.groupby(['gender','genre']).apply(lambda x: pd.Series({
        'prop_signif_per_gender_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change_value_per_gender_per_genre': x['slope_change'].mean(),
        'total_number_signif_per_gender_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_gender_by_genre_prop_df)

#Calculate average magnitude of slope change 
name_by_gender_by_genre_prop_magnitude_df = movie_genre_aggregate_df.groupby(['gender','genre']).apply(lambda x: pd.Series({
        'prop_signif_per_gender_per_genre': (x['p_value'] < 0.1).sum()/len(x['p_value']),
        'avg_slope_change_magnitude_per_gender_per_genre': x['slope_change'].abs().mean(),
        'total_number_signif_per_gender_per_genre': (x['p_value'] < 0.1).sum(),
    }))
display(name_by_gender_by_genre_prop_magnitude_df)

prop_signif_per_gender_per_genre  \
gender genre                                                     
F      Absurdism                                      0.156863   
       Acid western                                   0.166667   
       Action                                         0.157500   
       Action Comedy                                  0.178030   
       Action Thrillers                               0.181692   
...                                                        ...   
M      Women in prison films                          0.153846   
       Workplace Comedy                               0.141667   
       World cinema                                   0.079252   
       Wuxia                                          0.060976   
       Zombie Film                                    0.121891   

                              avg_slope_change_value_per_gender_per_genre  \
gender genre                                                                
F      Absurdism                                                -0.000930   
       Acid western                                             -0.001279   
       Action                                                    0.000176   
       Action Comedy                                             0.000645   
       Action Thrillers                                          0.000482   
...                                                                   ...   
M      Women in prison films                                    -0.008910   
       Workplace Comedy                                         -0.000440   
       World cinema                                             -0.000068   
       Wuxia                                                    -0.000335   
       Zombie Film                                              -0.000948   

                              total_number_signif_per_gender_per_genre  
gender genre                                                            
F      Absurdism                                                  32.0  
       Acid western                                                1.0  
       Action                                                   1366.0  
       Action Comedy                                              47.0  
       Action Thrillers                                          131.0  
...                                                                ...  
M      Women in prison films                                       2.0  
       Workplace Comedy                                           51.0  
       World cinema                                              691.0  
       Wuxia                                                       5.0  
       Zombie Film                                                49.0  

[655 rows x 3 columns]

prop_signif_per_gender_per_genre  \
gender genre                                                     
F      Absurdism                                      0.156863   
       Acid western                                   0.166667   
       Action                                         0.157500   
       Action Comedy                                  0.178030   
       Action Thrillers                               0.181692   
...                                                        ...   
M      Women in prison films                          0.153846   
       Workplace Comedy                               0.141667   
       World cinema                                   0.079252   
       Wuxia                                          0.060976   
       Zombie Film                                    0.121891   

                              avg_slope_change_magnitude_per_gender_per_genre  \
gender genre                                                                    
F      Absurdism                                                     0.003611   
       Acid western                                                  0.001279   
       Action                                                        0.004378   
       Action Comedy                                                 0.004563   
       Action Thrillers                                              0.005683   
...                                                                       ...   
M      Women in prison films                                         0.012976   
       Workplace Comedy                                              0.002512   
       World cinema                                                  0.002190   
       Wuxia                                                         0.000871   
       Zombie Film                                                   0.003009   

                              total_number_signif_per_gender_per_genre  
gender genre                                                            
F      Absurdism                                                  32.0  
       Acid western                                                1.0  
       Action                                                   1366.0  
       Action Comedy                                              47.0  
       Action Thrillers                                          131.0  
...                                                                ...  
M      Women in prison films                                       2.0  
       Workplace Comedy                                           51.0  
       World cinema                                              691.0  
       Wuxia                                                       5.0  
       Zombie Film                                                49.0  

[655 rows x 3 columns]

In [94]:
display(name_by_gender_by_genre_prop_magnitude_df['avg_slope_change_magnitude_per_gender_per_genre']['F']['Absurdism'])

display(name_by_gender_by_genre_prop_magnitude_df['avg_slope_change_magnitude_per_gender_per_genre']['M']['Absurdism'])

display(name_by_gender_by_genre_prop_magnitude_df['avg_slope_change_magnitude_per_gender_per_genre']['F'].idxmax())
display(name_by_gender_by_genre_prop_magnitude_df['avg_slope_change_magnitude_per_gender_per_genre']['M'].idxmax())


0.0036109347677933056

0.003890474545940062

'Outlaw biker film'

'Animals'